In [3]:
#-*- coding: utf-8 -*-

from multiprocessing import Process

from pandas import DataFrame
from pymongo import MongoClient
from firebase.firebase import FirebaseApplication

In [4]:
connection = MongoClient('localhost', 27018)
connection.database_names()

['admin', 'local', 'portal_transparencia']

In [5]:
db = connection['portal_transparencia']
db.collection_names()

['dados_abertos']

In [6]:
collection = db['dados_abertos']
collection.count()

38994

In [7]:
cursor = collection.find({})

In [8]:
df = DataFrame(list(cursor))

In [9]:
# df_2017 = df[(df.ano.apply(int) == 2017)][['estado', 'municipio', 'acoes']]
# df_2017.estado.drop_duplicates()

In [10]:
anos = df['ano'].drop_duplicates()
estados = df['estado'].drop_duplicates()

In [11]:
anos = sorted(anos)
estados = sorted(estados)

In [12]:
items = []
for estado in estados:
    _item = {}
    _item['estado'] = estado
    _municipios = sorted(df[(df.estado == estado)]['municipio'].drop_duplicates())

    municipios = []
    for municipio in _municipios:
        _mncp = {}
        _mncp['municipio'] = municipio

        municipios.append(_mncp)

    _item['municipios'] = municipios
    items.append(_item)

DB = {'estados': items}

In [13]:
# DB

In [14]:
# # Firebase Integration

# DSN = 'https://transparenciaeducativa-1e9ca.firebaseio.com'

# authentication = None
# firebase = FirebaseApplication(DSN, authentication)

In [15]:
# firebase.patch('/', DB)

In [16]:
# firebase.get('/',None)

In [17]:
# firebase.get('/','estados/0')

In [18]:
# firebase.get('/','estados/0/estado')

In [19]:
# firebase.get('/','estados/0/municipios')

In [20]:
# firebase.get('/','estados/0/municipios/0')

In [21]:
# governo_investe_municipio = {}
# governo_investe_municipio['despesas'] = {}
# for ano in anos:
#     acs = list(df[(df.estado == 'ACRE') & (df.municipio == 'ACRELANDIA') & (df.ano == ano)]['acoes'])
#     if acs:
#         governo_investe_municipio['despesas'][ano] = acs.pop()

In [22]:
# governo_investe_municipio

In [23]:
# firebase.patch('estados/0/municipios/0', governo_investe_municipio)

In [24]:
# firebase.get('/','estados/0/municipios/0')

In [25]:
# firebase.get('/','estados/0/municipios/0/despesas')

In [26]:
# firebase.get('/','estados/0/municipios/0/municipio')

In [27]:
def firebase_patch(lista):
    # Firebase Integration
    DSN = 'https://transparenciaeducativa-1e9ca.firebaseio.com'
    authentication = None
    firebase = FirebaseApplication(DSN, authentication)
    firebase.patch(lista[0],lista[1])

In [ ]:
for i, estado in enumerate(estados):   
    _municipios = sorted(df[(df.estado == estado)]['municipio'].drop_duplicates())
    for j, municipio in enumerate(_municipios):
        processos = []
        governo_investe_municipio = {}
        governo_investe_municipio['despesas'] = {}
        for ano in anos:
            acs = list(df[(df.estado == estado) & (df.municipio == municipio) & (df.ano == ano)]['acoes'])
            if acs:
                governo_investe_municipio['despesas'][ano] = acs.pop()
            _lista = ['estados/{e}/municipios/{m}'.format(e=i,m=j), governo_investe_municipio]
            processos.append(Process(target=firebase_patch, args=(_lista,)))
            
        for p in processos:
            p.start()
        for p in processos:
            p.join()

Process Process-8785:
Process Process-8779:
Process Process-8781:
Process Process-8784:
Process Process-8783:
Process Process-8780:
Process Process-8782:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 301, in _send_until_done
    return self.connection.send(data)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 301, in _send_until_done
    return self.connection.send(data)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 301, in _send_until_done
    return self.connection.send(data)
  File "/home/iury/

  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/iury/.pyenv/versi

Traceback (most recent call last):
OSError: (32, 'EPIPE')
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/connectionpool.py", line 649, in urlopen
    _stacktrace=sys.exc_info()[2])
OSError: (32, 'EPIPE')
OSError: (32, 'EPIPE')
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/connectionpool.py", line 649, in urlopen
    _stacktrace=sys.exc_info()[2])

During handling of the above exception, another exception occurred:

OSError: (32, 'EPIPE')
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/requests/adapters.py", line 440, in send
    timeout=timeout
Traceback (most recent call last):

During handling of the above exception, another exception occurred:

  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/util/retry.py", line 357, in increment
    raise six.reraise(type(error), error, 

  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/connectionpool.py", line 356, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/connectionpool.py", line 356, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 313, in sendall
    sent = self._send_until_done(data[total_sent:total_sent + SSL_WRITE_BLOCKSIZE])
  File "/home/iury/.pyenv/versions/3.6.1/lib

  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/firebase/decorators.py", line 19, in wrapped
    return f(*args, **kwargs)
Traceback (most recent call last):
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
urllib3.exceptions.ProtocolError: ('Connection aborted.', OSError("(32, 'EPIPE')",))
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/firebase/firebase.py", line 355, in patch
    connection=connection)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/firebase/decorators.py", line 19, in wrapped
    return f(*args, **kwargs)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py", line 313, in sendall
    sent = self._send_until_done(data[total_sent:total_sent + SSL_WRITE_BLOCKSIZE])

During handling of th

  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/firebase/decorators.py", line 19, in wrapped
    return f(*args, **kwargs)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/firebase/decorators.py", line 19, in wrapped
    return f(*args, **kwargs)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/requests/sessions.py", line 502, in request
    resp = self.send(prep, **send_kwargs)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/requests/sessions.py", line 612, in send
    r = adapter.send(request, **kwargs)
requests.exceptions.ConnectionError: ('Connection aborted.', OSError("(32, 'EPIPE')",))
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/firebase/firebase.py", line 355, in patch
    connection=connection)
  File "/home/iury/.pyenv/versions/3.6.1/envs/porta

  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/packages/six.py", line 685, in reraise
    raise value.with_traceback(tb)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/iury/.pyenv/versions/3.6.1/envs/portal_transparencia/lib/python3.6/site-packages/urllib3/connectionpool.py", line 356, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/home/iury/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encod